Music Genre Classification using Song Lyrics ||
Author: Parsa Keyvani - Sandra Weinstock ||

I affirm that I have carried out my academic endeavors with full
academic honesty. [Parsa Keyvani, Sandra Weinstock]


In [100]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

#from google.colab import drive
#drive.mount('/content/drive')

In [101]:
# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/NLP Group Project /preprocessed_data.csv")
data = data.dropna()

# Preprocessing
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['Lyrics'])
sequences = tokenizer.texts_to_sequences(data['Lyrics'])
max_length = max([len(x) for x in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Encode the target variable y
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(data['Genres'])

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

In [102]:
data.describe()


Genres
hip hop    263.0
pop        144.0
rock       104.0
Name: lyric_length, dtype: float64


In [ ]:
#### VISUALIZATION: This table documents the average lyric lengths for each of the genres
# compute the length of each lyric and add it as a new column
data['lyric_length'] = data['Lyrics'].apply(len)

# group the dataframe by genre and compute the mean of the lyric lengths for each group
avg_lyric_length_by_genre = data.groupby('Genres')['lyric_length'].mean()
# print the resulting table
print(round(avg_lyric_length_by_genre/10))

In [ ]:
#### VISUALIZATION: This table shows the top 10 unique words mentioned in each genre the most that is different in other genres. 

# Loop through each genre
from collections import Counter


for genre in set(data["Genres"]):
    genre_words = []
    other_words = []
    
    # Get all the words for the current genre
    for lyric in data[data["Genres"] == genre]["Lyrics"]:
        genre_words += lyric.split()
    
    # Get all the words for the other genres
    for other_genre in set(data["Genres"]) - {genre}:
        for lyric in data[data["Genres"] == other_genre]["Lyrics"]:
            other_words += lyric.split()
    
    # Count the occurrences of each word in the current genre
    genre_word_counts = Counter(genre_words)
    

    
   # Print the top 10 unique words for the current genre
    print("Genre:", genre)
    for word, count in genre_word_counts.most_common()[-10:]:
        print(word)
    print()

In [ ]:
#### VISUALIZATION: This plot shows the confusion matrix for the logistic regression model using average GloVe
# embeddings. The rows represent true genre while the columns represent predicted genre.

cm = confusion_matrix(y_test, y_pred)

# plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

In [ ]:
# Load the pre-trained GloVe embeddings
embeddings_index = {}
with open("/content/drive/MyDrive/NLP Group Project /glove.6B.100d.txt") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create the embedding matrix
embedding_dim = 100
word_index = tokenizer.word_index
num_words = min(len(word_index) + 1, 5000)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Train logistic regression model
clf = LogisticRegression(max_iter=500)
clf.fit(X_train, y_train)

# Make predictions on testing set
y_pred = clf.predict(X_test)

# Evaluate model performance
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print('F1 score:', f1)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

# Train LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(num_words, embedding_dim, input_length=max_length))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(10, activation='softmax'))
lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Train bidirectional LSTM model
bilstm_model = Sequential()
bilstm_model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
bilstm_model.add(Bidirectional(LSTM(units=32, dropout=0.2)))
bilstm_model.add(Dense(10, activation='softmax'))
bilstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = bilstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 score: 0.6058531008241017
Accuracy: 0.645996387718242
Precision: 0.6091903458895656
Recall: 0.645996387718242
Epoch 1/10
104/104 [==============================] - 1179s 11s/step - loss: 1.0608 - accuracy: 0.5543 - val_loss: 0.9576 - val_accuracy: 0.5671
Epoch 2/10
 37/104 [=========>....................] - ETA: 11:52 - loss: 0.9568 - accuracy: 0.5524